# Import Modules

In [1]:
import numpy as np
from ase.io.cube import read_cube_data, read_cube, write_cube
import plotly as py
import plotly.graph_objs as go
import copy
import random

# Script Inputs

In [2]:
# master_data_filter = 0.96
master_data_filter = 0.98
density_filter = 0.025

file_name = "data/Fe_graph_density.cube"

# Load Data

In [3]:
with open(file_name, "r") as fle:
    data_master = read_cube(fle)
    
atoms = data_master["atoms"]
data = data_master["data"]
origin = data_master["origin"]

In [4]:
cell_orig = copy.deepcopy(atoms.cell)

# atoms.positions[0]

array([ 3.90031227,  0.61413979, 12.00000026])

In [5]:
x_sum = cell_orig[0, :].sum()
y_sum = cell_orig[1, :].sum()
z_sum = cell_orig[2, :].sum()

x_size = data.shape[0]
y_size = data.shape[1]
z_size = data.shape[2]

In [6]:
x_list = []
y_list = []
z_list = []

dens_list = []
for index, dens_i in np.ndenumerate(data):
    x_list.append(index[0])
    y_list.append(index[1])
    z_list.append(index[2])
    
    dens_list.append(dens_i)
    
x_list = np.array(x_list)
y_list = np.array(y_list)
z_list = np.array(z_list)
dens_list = np.array(dens_list)

In [7]:
print("Number of data points:")
print(len(x_list));print("\n")

print("Mean Density:")
print(dens_list.mean());print("\n")

print("Max Density:")
print(dens_list.max());print("\n")

Number of data points:
1769472


Mean Density:
0.056685123016029355


Max Density:
3.709




In [8]:
bool_list = []
for data_i in range(len(x_list)):
    rand_i = random.random()   
    if rand_i > master_data_filter:
        bool_list.append(True)
    else:
        bool_list.append(False)
        
x_list = x_list[bool_list]
y_list = y_list[bool_list]
z_list = z_list[bool_list]
dens_list = dens_list[bool_list]

# Normalizing electron density
dens_list /= dens_list.max()

print(len(x_list))

352617


In [9]:
dens_boo_list = []
for dens_i in dens_list:
    if dens_i < density_filter:
        dens_boo_list.append(False)
    else:
        dens_boo_list.append(True)
        
x_list = x_list[dens_boo_list]
y_list = y_list[dens_boo_list]
z_list = z_list[dens_boo_list]
dens_list = dens_list[dens_boo_list]

print(len(x_list))

33309


In [10]:
atoms.cell[0] /= np.linalg.norm(atoms.cell[0])
atoms.cell[1] /= np.linalg.norm(atoms.cell[1])
atoms.cell[2] /= np.linalg.norm(atoms.cell[2])

In [11]:
atoms.cell

array([[1.        , 0.        , 0.        ],
       [0.50000081, 0.86602494, 0.        ],
       [0.        , 0.        , 1.        ]])

In [12]:
len(x_list)

x_list = x_list.astype(float)
y_list = y_list.astype(float)
z_list = z_list.astype(float)

x_list_new = []
y_list_new = []
z_list_new = []
for x_i, y_i, z_i in zip(x_list, y_list, z_list):
    b = np.array([x_i / x_size, y_i / y_size, z_i / z_size])
    new_coord = atoms.cell.dot(b)

    x_i_new = new_coord[0] * ((x_i + 1) / x_size) * x_sum
    y_i_new = new_coord[1] * ((y_i + 1) / y_size) * y_sum
    z_i_new = new_coord[2] * ((z_i + 1) / z_size) * z_sum

    x_list_new.append(x_i_new)
    y_list_new.append(y_i_new)
    z_list_new.append(z_i_new)
    
    
x_list = x_list_new
y_list = y_list_new
z_list = z_list_new

In [13]:
print(x_i / x_size)

print(new_coord[0])

x_i

# atoms.cell
x_i_new

x_i
x_size

x_sum
y_sum
z_sum

z_i_new

0.984375
0.984375


12.406373078506329

# Atoms

In [14]:
atoms.positions[:, 2]

# atoms[0]

color_list = []
for atom in atoms:
    elem = atom.symbol
    print(elem)
    if elem == "Fe":
        color_list.append("orange")
    elif elem == "C":
        color_list.append("grey")
        


Fe
Fe
Fe
Fe
Fe
Fe
Fe
Fe
Fe
C
C
C
C
C
C


In [15]:
trace2 = go.Scatter3d(
    x=atoms.positions[:, 0],
    y=atoms.positions[:, 1],
    z=atoms.positions[:, 2],
    mode='markers',

    marker=dict(
        size=18,
        color=color_list,
#         color=dens_list,  # set color to an array/list of desired values
       
    )
)

# Unit Cell

In [16]:
line_width = 1
color = "red"

In [17]:
line_i = np.array([
    np.array([0., 0., 0.]),
    cell_orig[0],
    ])

trace_a = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]),
    cell_orig[1],
    ])
trace_b = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)


line_i = np.array([
    np.array([0., 0., 0.]),
    cell_orig[2],
    ])
trace_c = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    cell_orig[0],
    cell_orig[0] + cell_orig[1],
    ])
trace_d = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    cell_orig[0],
    cell_orig[0] + cell_orig[2],
    ])
trace_e = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    cell_orig[1],
    cell_orig[1] + cell_orig[0],
    ])
trace_f = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    cell_orig[1],
    cell_orig[1] + cell_orig[2],
    ])
trace_g = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]) + cell_orig[2],
    np.array([0., 0., 0.]) + cell_orig[2] + cell_orig[0],
    ])
trace_h = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]) + cell_orig[2],
    np.array([0., 0., 0.]) + cell_orig[2] + cell_orig[1],
    ])
trace_i = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]) + cell_orig[0] + cell_orig[1] + cell_orig[2],
    np.array([0., 0., 0.]) + cell_orig[2] + cell_orig[1],
    ])
trace_j = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]) + cell_orig[0] + cell_orig[1] + cell_orig[2],
    np.array([0., 0., 0.]) + cell_orig[0] + cell_orig[1],
    ])
trace_k = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)

line_i = np.array([
    np.array([0., 0., 0.]) + cell_orig[0] + cell_orig[1] + cell_orig[2],
    np.array([0., 0., 0.]) + cell_orig[0] + cell_orig[2],
    ])
trace_l = go.Scatter3d(
    x=line_i[:, 0], y=line_i[:, 1], z=line_i[:, 2],
    marker=dict(
        size=0,
        colorscale='Viridis',
    ),
    line=dict(
        color=color,
        width=line_width
    )
)



line_trace_list = [
    trace_a,
    trace_b,
    trace_c,
    trace_d,
    trace_e,
    trace_f,
    trace_g,
    trace_h,
    trace_i,
    trace_j,
    trace_k,
    trace_l,
    ]

In [18]:
trace1 = go.Scatter3d(
    x=x_list,
    y=y_list,
    z=z_list,
    mode='markers',

    marker=dict(
        size=2,
        color=dens_list,  # set color to an array/list of desired values

        colorscale= [
            [0.0, 'rgb(255, 229.5, 229.5, 0.9)'],
            [0.1, 'rgb(255, 229.5, 229.5, 0.9)'],

            [0.1, 'rgb(255, 204.0, 204.0, 0.9)'],
            [0.2, 'rgb(255, 204.0, 204.0, 0.9)'],

            [0.2, 'rgb(255, 178.5, 178.5, 0.9)'],
            [0.3, 'rgb(255, 178.5, 178.5, 0.9)'],

            [0.3, 'rgb(255, 153.0, 153.0, 0.9)'],
            [0.4, 'rgb(255, 153.0, 153.0, 0.9)'],

            [0.4, 'rgb(255, 127.0, 127.0, 0.9)'],
            [0.5, 'rgb(255, 127.0, 127.0, 0.9)'],

            [0.5, 'rgb(255, 102.0, 102.0, 0.9)'],
            [0.6, 'rgb(255, 102.0, 102.0, 0.9)'],

            [0.6, 'rgb(255, 76.5, 76.5, 0.9)'],
            [0.7, 'rgb(255, 76.5, 76.5, 0.9)'],

            [0.7, 'rgb(255, 51, 51, 0.9)'],
            [0.8, 'rgb(255, 51, 51, 0.9)'],

            [0.8, 'rgb(255, 25.5, 25.5, 0.9)'],
            [0.9, 'rgb(255, 25.5, 25.5, 0.9)'],

            [0.9, 'rgb(255, 0, 0, 0.9)'],
            [1.0, 'rgb(255, 0, 0, 0.9)'],
        ],
        
    )
)

In [20]:

data = [
    trace1,
    trace2,
    ]

data += line_trace_list

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)

py.plotly.image.save_as(fig, filename='a-simple-plot.png')

# py.plotly.iplot(fig, filename='__temp__/charge_den')